In [8]:
search_for = 89
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004179954528808594
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 63455221
type: [1, 1, 1, 1, 89] 89
cases of this type: 62742241
10000 9.368703516542137
20000 9.16748576289918
30000 28.405528906239272
40000 36.77269626211991
50000 36.87316321416
60000 36.235259425423145
70000 39.688047784611484
80000 36.19921984526852
90000 37.54230031196586
100000 35.88340263967836
110000 36.636625691220175
120000 35.46159403078857
130000 35.34061258254908
140000 35.717333070716265
150000 36.651475097286244
160000 36.024770820370236
170000 36.901776305966926
180000 40.79331732275447
190000 39.38451626004541
200000 36.16631159192132
210000 34.93244886919004
220000 36.277258386982
230000 35.5393187886065
240000 35.10372478992289
250000 33.51209160508442
260000 36.31071053271423
270000 34.53967750963129
280000 36.476811261963306
290000 33.82930531296056
300000 36.69800031312279
310000 33.23428244748153
320000 36

3080000 30.583608840491117
3090000 31.0657928496887
3100000 36.47527073716783
3110000 30.922245134522765
3120000 33.313295771778726
3130000 32.15605945570737
3140000 30.304617196778143
3150000 31.73307955068722
3160000 13.87240710229981
3170000 7.094631903738671
3180000 7.106691302481969
3190000 5.339706108083407
3200000 30.863631918904417
3210000 30.248844423740422
3220000 31.24983879971975
3230000 30.246008646191516
3240000 31.685713667668136
3250000 35.5939439218027
3260000 31.775362100378402
3270000 31.285952001235298
3280000 29.4234626421748
3290000 31.111283351823776
3300000 29.93632704984611
3310000 30.244229282598774
3320000 33.729387424409346
3330000 33.51444414079908
3340000 32.66510076748556
3350000 34.74215099127668
3360000 31.54372714211641
3370000 33.176597020803825
3380000 35.094246315739944
3390000 29.683259477188695
3400000 29.926732645950043
3410000 32.53006756862125
3420000 29.91255568284332
3430000 29.938482037723066
3440000 33.3143174514413
3450000 31.0667453397251

6180000 30.53126895299218
6190000 29.004035679003216
6200000 30.366985664969928
6210000 32.27892718790815
6220000 30.56448018061334
6230000 29.522200978319017
6240000 30.38881672479695
6250000 31.124514372046892
6260000 28.51013979088962
6270000 12.210169284053576
6280000 12.50940946735698
6290000 12.88094356622827
6300000 20.52711133622184
6310000 34.94897574207569
6320000 31.99112000541847
6330000 27.308003681790044
6340000 34.422452991391935
6350000 32.11079825971633
6360000 35.482394946758056
6370000 29.900749675357456
6380000 31.484787462005716
6390000 30.817026304591174
6400000 35.55330864307505
6410000 10.86241602046026
6420000 10.03977926117827
6430000 10.022714186330298
6440000 11.299335779766473
6450000 34.93836208515933
6460000 29.874923863003573
6470000 29.94769872078664
6480000 31.76521756968075
6490000 31.52752879275924
6500000 30.121250258946507
6510000 29.654448196873744
6520000 29.473183895121153
6530000 30.083520341434202
6540000 30.176181303422492
6550000 30.49422715

9270000 9.407843630053037
9280000 9.279569212846578
9290000 9.443250888542858
9300000 32.76400657501018
9310000 29.290604616228602
9320000 26.429381122262818
9330000 28.54235935461201
9340000 29.613038661270757
9350000 27.52654152414893
9360000 26.961190311683737
9370000 27.925760035668254
9380000 28.7386927050428
9390000 28.21099643544464
9400000 27.28129656662017
9410000 29.29660960792034
9420000 26.652074324266945
9430000 24.85966340883259
9440000 24.869451194770004
9450000 25.77613375405501
9460000 25.60105387033814
9470000 24.520612030294146
9480000 25.57127186455835
9490000 27.06445417844647
9500000 22.80823591148178
9510000 4.860074830819726
9520000 4.8874619091041085
9530000 4.840512769719654
9540000 14.740642556334715
9550000 27.768037449182273
9560000 24.912344592290665
9570000 25.35461196641265
9580000 26.923414082316317
9590000 24.272532322004125
9600000 24.773467201501912
9610000 25.73570904830704
9620000 25.845641556077236
9630000 25.621884465603305
9640000 28.16469596586

12270000 24.011685404824863
12280000 22.662381995779576
12290000 22.667583833073014
12300000 22.50438344319849
12310000 25.45010004523017
12320000 20.7904676209741
12330000 4.585532288699448
12340000 4.502524985622764
12350000 4.486478230124851
12360000 13.536514263317647
12370000 26.07441733782899
12380000 22.914862849480166
12390000 24.11939203277125
12400000 23.309051278814614
12410000 22.300463356715838
12420000 22.885623181730136
12430000 23.530160894281625
12440000 23.613556661407205
12450000 24.734438043890716
12460000 25.154850349084615
12470000 24.783882405579106
12480000 23.988496534209187
12490000 24.098023707052928
12500000 26.33910484716142
12510000 25.03373174107176
12520000 24.40175227609706
12530000 27.74794046219554
12540000 13.473866219549047
12550000 10.892486419830103
12560000 10.659033282137123
12570000 10.816091368897451
12580000 31.16125002201603
12590000 28.873451709969657
12600000 25.06373729296599
12610000 26.60239651361227
12620000 28.347947376795467
12630000

...

51150000 5.451097491483052
51160000 4.870376899153014
51170000 5.014623877596702
51180000 4.732787983392888
51190000 4.75808417227006
51200000 4.509971860808809
51210000 5.102799862752094
51220000 4.841032711969197
51230000 5.156992888740579
51240000 5.167542534519116
51250000 5.990111049515426
51260000 5.538552512502565
51270000 5.7525402599618305
51280000 5.366811669360875
51290000 6.0486575203563575
51300000 5.528555755842712
51310000 5.764530460495949
51320000 3.1506637866802483
51330000 2.0407423026847247
51340000 1.8197017339804766
51350000 2.048979313758234
51360000 5.7215640897002285
51370000 5.732617803282917
51380000 4.421070447939111
51390000 5.129097906080147
51400000 4.739858054004907
51410000 5.710405227991932
51420000 5.306261682111469
51430000 6.017625344345708
51440000 5.695026446424437
51450000 5.467024313498531
51460000 5.274162044606586
51470000 5.436838621775442
51480000 5.936573650758604
51490000 3.6881117366340166
51500000 4.1876269114599225
51510000 4.9972881242

54170000 2.5460098338117203
54180000 1.5302072337657942
54190000 1.5130112424769997
54200000 1.3555336123591464
54210000 4.094702399004983
54220000 3.6382751794429415
54230000 4.10810879260393
54240000 4.293170161971555
54250000 3.9504684354435406
54260000 4.004577809295575
54270000 3.855466357990583
54280000 4.564032552237868
54290000 4.331057184766054
54300000 4.449008140056153
54310000 4.4681571320541895
54320000 3.1738862898051474
54330000 4.309954463655916
54340000 3.9946735719996096
54350000 4.539039708644039
54360000 3.1300727226371237
54370000 1.7071895020990968
54380000 1.783497086510148
54390000 1.598480896784021
54400000 3.98728224303027
54410000 4.638857582432059
54420000 4.2054750482106344
54430000 4.037559139131208
54440000 4.267547724331929
54450000 4.103440296121373
54460000 4.188873395347535
54470000 4.1082333202296235
54480000 4.037686745486286
54490000 3.7272427175797427
54500000 3.9850005692348414
54510000 4.3517803329093185
54520000 4.144470075841685
54530000 3.084

57160000 0.8159983180071712
57170000 0.7764937107315064
57180000 1.3562913906638359
57190000 3.066875591830333
57200000 3.038503011712657
57210000 3.0006032514838377
57220000 3.0978205259880025
57230000 3.0174249617958933
57240000 2.6375547194117837
57250000 2.597524966945231
57260000 3.1254906254978243
57270000 1.929317363163564
57280000 2.8194718719462153
57290000 2.1163682230790326
57300000 1.957084254299541
57310000 2.805890472257952
57320000 2.798510818447769
57330000 2.8173040194609236
57340000 3.110718801324129
57350000 2.5237861651263964
57360000 2.039525523293469
57370000 2.4817189551228482
57380000 1.4310252245191137
57390000 2.084402470630639
57400000 2.106028139610668
57410000 2.315230300770018
57420000 2.49573729823237
57430000 2.919743292753398
57440000 1.4198533773517872
57450000 0.2869502029701074
57460000 0.3207599471720259
57470000 0.36791061495116023
57480000 1.593853624575118
57490000 3.0693881718447207
57500000 2.394875373869684
57510000 2.5055137692747445
57520000

60120000 1.5329169552336268
60130000 1.2169891210590005
60140000 1.4872538114193548
60150000 1.512314755848474
60160000 1.4311246024405162
60170000 1.1095479212704764
60180000 0.9459208766593403
60190000 1.5284449702247183
60200000 1.413935122985575
60210000 1.3610719613826405
60220000 1.5367565132709144
60230000 1.4903685446239776
60240000 1.041413566345281
60250000 1.6456800297073324
60260000 0.7263513571023875
60270000 0.19569545635392266
60280000 0.16851920516721408
60290000 0.2662274774486489
60300000 1.0081966374809146
60310000 1.7034825288294553
60320000 0.6756918390268817
60330000 0.9072344930215875
60340000 1.2818237684907714
60350000 1.5870270429355304
60360000 1.1142875168292257
60370000 1.176689239086469
60380000 0.7898873493872617
60390000 0.7207270949337482
60400000 0.7921738256963492
60410000 0.8958204326598379
60420000 1.1496986818209225
60430000 0.9130563744133313
60440000 1.4232406132929392
60450000 1.1420558943611514
60460000 1.5122127904894749
60470000 1.20942461455

63010000 0.031222905196249484
63020000 0.03058941390770674
63030000 0.030394254701972004
63040000 0.029244106035411357
63050000 0.02842040993159347
63060000 0.027743477511598003
63070000 0.026885609022796155
63080000 0.026079299283610448
63090000 0.025593275907185343
63100000 0.024731714040637016
63110000 0.02391371068775654
63120000 0.02347357089781099
63130000 0.02288778071067731
63140000 0.021945147409770223
63150000 0.021073137163897353
63160000 0.0205926416631341
63170000 0.019997398670044207
63180000 0.019066660679929785
63190000 0.0184275555909276
63200000 0.01772070738488436
63210000 0.01729569632538822
63220000 0.01660894702744484
63230000 0.01599144272039334
63240000 0.015115227511558268
63250000 0.014389425828138985
63260000 0.013497963377018769
63270000 0.013408932036976018
63280000 0.012482599204480648
63290000 0.011573361394259666
63300000 0.01098402264112234
63310000 0.010232688338736693
63320000 0.009514177088836829
63330000 0.00871473714438412
63340000 0.00805601158056